In [8]:
###############################
## Tabellen vullen
###############################
import os, sys, inspect, json
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)
from lib.Utilities.globalVar import *
import lib.TransformData.combineData as combd
import Download.download as download
from Utilities.converter import *

currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)
import urllib
from sqlalchemy import *
from sqlalchemy.exc import IntegrityError, InternalError
from sqlalchemy.types import BINARY, VARCHAR
import pandas as pd
import numpy as np
from math import *


import pyodbc


from sqlalchemy.types import TypeDecorator, CHAR
from sqlalchemy.dialects.postgresql import UUID
import uuid

In [9]:

server = 'sic-match-analysis.database.windows.net'
database = 'match-analysis'
username = 'sic-admin'
password = 'ZeilenIsLeuk!'
driver= '{ODBC Driver 17 for SQL Server}'
connection_str = 'DRIVER='+driver+';SERVER='+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password


cnxn = pyodbc.connect(connection_str)
cursor = cnxn.cursor()

quoted = urllib.parse.quote_plus(connection_str)
engine = create_engine('mssql+pyodbc:///?odbc_connect={}'.format(quoted))

from sqlalchemy import event

@event.listens_for(engine, 'before_cursor_execute')
def receive_before_cursor_execute(conn, cursor, statement, params, context, executemany):
    if executemany:
        cursor.fast_executemany = True
        cursor.commit()

In [10]:
## Lukt niet om met bulk insert vanaf locale dir bestanden up te loaden :( dus moet via pandas to_sql
# cursor.execute("""
# BULK INSERT regattas
# FROM '\\usman-ess/Folder Redirection/Nerine/Documents/TU_Delft/2_Master/3_Stage/2_Code/SAPDataAnalysis/data/raw/tokyo2019'
# """)
event  = [{'name' :  'hwcs2020-round1', 'AllRegattas': True, 'regattaNameContaining': '', 'raceNameContaining': ''}
         ,{'name' : 'www', 'AllRegattas' : False, 'regattaNameContaining': 'Enoshima', 'raceNameContaining': ''}
         ]
event = [{'name' :  'tokyo2019', 'AllRegattas': True, 'regattaNameContaining': '', 'raceNameContaining': ''}]
eventSelection = event
parentdir

'X:\\TU_Delft\\2_Master\\3_Stage\\2_Code\\SAPDataAnalysis'

In [11]:
## TABLE 1 regattas
for serv in eventSelection:
    filepath = parentdir + '/' +  outdirbase + serv['name'] + '/regattas'
    with open(filepath + '.json') as json_file:
        print('file extracted: ', filepath)
        data = json.load(json_file)
    cd = combd.combineData(serv)
    df = cd.transformJsonToPD(data, 'regattas')
    serv['regattas'] = df
    try:
        df.to_sql('regattas', con = engine, index = False, if_exists = 'append')
        print('Succesfully added regattas of {} to the database'.format(serv['name']))
    except IntegrityError as e:
        print('Could not add regattas of {} to the database because of IntegrityError: {}'.format(serv['name'], e))
    


file extracted:  X:\TU_Delft\2_Master\3_Stage\2_Code\SAPDataAnalysis//data/raw/tokyo2019/regattas
Could not add regattas of tokyo2019 to the database because of IntegrityError: (pyodbc.IntegrityError) ('23000', "[23000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Violation of PRIMARY KEY constraint 'PK__regattas__B87EB5247FB8C6A6'. Cannot insert duplicate key in object 'dbo.regattas'. The duplicate key value is (Tokyo 2019 - 49er). (2627) (SQLExecute); [23000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]The statement has been terminated. (3621); [23000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Violation of PRIMARY KEY constraint 'PK__regattas__B87EB5247FB8C6A6'. Cannot insert duplicate key in object 'dbo.regattas'. The duplicate key value is (Tokyo 2019 - Laser). (2627); [23000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]The statement has been terminated. (3621); [23000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Violation of PRIMA

In [5]:
# TABLE 2 races columns; 
# regatta
# race
# raceShort
# raceId
# maxWindSpd_kts
# minWindSpd_kts
# avgWindDir_deg
# firstlegbearing_deg

# coming from races, windsummary and firstlegbearing
filenames = ['races', 'windsummary', 'firstlegbearing']

for serv in eventSelection:
    for indx, regatta in serv['regattas'].iterrows():
        regName = regatta.regatta
        directory_regatta = parentdir + '/' + outdirbase + webAddressToFilename(serv['name']) + \
                            webAddressToFilename(regattaLoc(regName))
        filepath = directory_regatta + 'races.json'
        with open(filepath) as json_file:
            print('file extracted: ', filepath)
            data = json.load(json_file)
        dic_data = {'regatta': regName, 'data': data}
        races = cd.transformJsonToPD(dic_data, 'races')
        
        
        filepath = directory_regatta + 'windsummary.json'
        with open(filepath) as json_file:
            print('file extracted: ', filepath)
            data = json.load(json_file)
        dic_data = {'regatta': regName, 'data': data}
        windsummary = cd.transformJsonToPD(dic_data, 'windsummary').set_index(['regatta', 'raceShort'])   
        
        flb = []
        for index, race in races.iterrows():
            directory_race = parentdir + '/' + outdirbase +serv['name'] + \
                             webAddressToFilename(raceLoc(regName, race['race']))
            filepath = directory_race + 'firstlegbearing.json'
            with open(filepath) as json_file:
#                 print('file extracted: ', filepath)
                data = json.load(json_file)
            dic_data = {'regatta': regName, 'race': race['race'], 'data': data}
            firstlegbearing = cd.transformJsonToPD(dic_data, 'firstlegbearing') 
            flb.append(firstlegbearing)
        flb = pd.concat(flb).set_index(['regatta', 'race'])
        
    
        races = races.set_index(['regatta','raceShort'])
        result = pd.concat([races, windsummary], axis=1, join='outer').reset_index().set_index(['regatta', 'race'])
        result = pd.concat([result, flb], axis=1, join='outer').reset_index()
        
#         print('with as result', result)
        
        try:
            result.to_sql('races', con = engine, index = False, if_exists = 'append')
            print('Succesfully added races of {} {} to the database'.format(regName, race['race']))
        except IntegrityError as e:
            print('Could not add races of {} {} to the database because of IntegrityError: {}'.format(regName,race['race'],e))
    





file extracted:  X:\TU_Delft\2_Master\3_Stage\2_Code\SAPDataAnalysis//data/raw/hwcs2020-round1/regattas/HWCS 2020 Round 1 - Laser/races.json
file extracted:  X:\TU_Delft\2_Master\3_Stage\2_Code\SAPDataAnalysis//data/raw/hwcs2020-round1/regattas/HWCS 2020 Round 1 - Laser/windsummary.json
Could not add races of HWCS 2020 Round 1 - Laser R10 (Laser Standard) to the database because of IntegrityError: (pyodbc.IntegrityError) ('23000', "[23000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Cannot insert duplicate key row in object 'dbo.races' with unique index 'idx_rac'. The duplicate key value is (HWCS 2020 Round 1 - Laser, M Medal (Laser Standard)). (2601) (SQLExecute); [23000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]The statement has been terminated. (3621)")
[SQL: INSERT INTO races (regatta, race, [raceShort], [raceId], [minWindSpd_kts], [maxWindSpd_kts], [avgWindDir_deg], firstlegbearing_deg) VALUES (?, ?, ?, ?, ?, ?, ?, ?)]
[parameters: (('HWCS 2020 Round 1 - Las

Could not add races of HWCS 2020 Round 1 - Nacra 17 M Medal (Nacra 17) to the database because of IntegrityError: (pyodbc.IntegrityError) ('23000', "[23000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Cannot insert duplicate key row in object 'dbo.races' with unique index 'idx_rac'. The duplicate key value is (HWCS 2020 Round 1 - Nacra 17, M Medal (Nacra 17)). (2601) (SQLExecute); [23000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]The statement has been terminated. (3621); [23000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Cannot insert duplicate key row in object 'dbo.races' with unique index 'idx_rac'. The duplicate key value is (HWCS 2020 Round 1 - Nacra 17, R1 (Nacra 17)). (2601); [23000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]The statement has been terminated. (3621); [23000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Cannot insert duplicate key row in object 'dbo.races' with unique index 'idx_rac'. The duplicate key value 

Could not add races of HWCS 2020 Round 1 - RS:X Men R4 (RSX Men) to the database because of IntegrityError: (pyodbc.IntegrityError) ('23000', "[23000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Cannot insert duplicate key row in object 'dbo.races' with unique index 'idx_rac'. The duplicate key value is (HWCS 2020 Round 1 - RS:X Men, M Medal (RSX Men)). (2601) (SQLExecute); [23000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]The statement has been terminated. (3621); [23000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Cannot insert duplicate key row in object 'dbo.races' with unique index 'idx_rac'. The duplicate key value is (HWCS 2020 Round 1 - RS:X Men, R1 (RSX Men)). (2601); [23000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]The statement has been terminated. (3621); [23000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Cannot insert duplicate key row in object 'dbo.races' with unique index 'idx_rac'. The duplicate key value is (HWCS

Could not add races of HWCS 2020 Round 1 - RS:X Women R8 (RSX Women) to the database because of IntegrityError: (pyodbc.IntegrityError) ('23000', "[23000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Cannot insert duplicate key row in object 'dbo.races' with unique index 'idx_rac'. The duplicate key value is (HWCS 2020 Round 1 - RS:X Women, M Medal (RSX Women)). (2601) (SQLExecute); [23000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]The statement has been terminated. (3621); [23000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Cannot insert duplicate key row in object 'dbo.races' with unique index 'idx_rac'. The duplicate key value is (HWCS 2020 Round 1 - RS:X Women, R1 (RSX Women)). (2601); [23000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]The statement has been terminated. (3621); [23000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Cannot insert duplicate key row in object 'dbo.races' with unique index 'idx_rac'. The duplicate key va

Could not add races of HWCS 2020 Round 1 - 470 Women R09 (470 Women) to the database because of IntegrityError: (pyodbc.IntegrityError) ('23000', "[23000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Cannot insert duplicate key row in object 'dbo.races' with unique index 'idx_rac'. The duplicate key value is (HWCS 2020 Round 1 - 470 Women, M Medal (470 Women)). (2601) (SQLExecute); [23000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]The statement has been terminated. (3621); [23000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Cannot insert duplicate key row in object 'dbo.races' with unique index 'idx_rac'. The duplicate key value is (HWCS 2020 Round 1 - 470 Women, R08 (470 Women)). (2601); [23000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]The statement has been terminated. (3621); [23000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Cannot insert duplicate key row in object 'dbo.races' with unique index 'idx_rac'. The duplicate key val

Could not add races of WCS 2019 Enoshima - RS:X Men R1 to the database because of IntegrityError: (pyodbc.IntegrityError) ('23000', "[23000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Cannot insert duplicate key row in object 'dbo.races' with unique index 'idx_rac'. The duplicate key value is (WCS 2019 Enoshima - RS:X Men, M Medal (RS:X Men)). (2601) (SQLExecute); [23000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]The statement has been terminated. (3621); [23000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Cannot insert duplicate key row in object 'dbo.races' with unique index 'idx_rac'. The duplicate key value is (WCS 2019 Enoshima - RS:X Men, R1). (2601); [23000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]The statement has been terminated. (3621); [23000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Cannot insert duplicate key row in object 'dbo.races' with unique index 'idx_rac'. The duplicate key value is (WCS 2019 Enoshima - RS:

KeyboardInterrupt: 

In [ ]:
## TABLE 3 Competitors


for serv in eventSelection:
    d = download.download(server = serv['name'], regattasContaining = serv['regattaNameContaining'],\
    racesContaining = serv['raceNameContaining'])
    for indx, regatta in serv['regattas'].iterrows():
        regName = regatta.regatta
        
        directory_regatta =  parentdir + '/' + outdirbase + webAddressToFilename(serv['name']) +\
                                webAddressToFilename(regattaLoc(regName))
        filepath = directory_regatta + 'entries.json'
        with open(filepath) as json_file:
#             print('file extracted: ', filepath)
            data = json.load(json_file)
        dic_data = {'regatta': regName, 'data': data}
        competitors = cd.transformJsonToPD(dic_data, 'entries').set_index(['regatta', 
                                                                          'comp_name'])
#         print(competitors.head())


        dir_datamining =  parentdir + '/' + outdirbase + webAddressToFilename(serv['name']) + \
                        webAddressToFilename(regattaLoc(regName)) + '/datamining/'
        datamining_reg = []
        for root, dirs, files in os.walk(dir_datamining, topdown=True):
            for file in files:
                fname = os.path.splitext(file)[0]
                if file.endswith(".json"):
                    with open(root + file) as json_file:
#                         print('file extracted: ', root + file)
                        data = json.load(json_file)
                    dic_data = {'regatta': regName, 'data': data}
                    df = cd.transformJsonToPD(dic_data, fname).set_index(['regatta', 'comp_name']) 
                    datamining_reg.append(df)
#                   print('df: ',df.head())

        datamining_reg = pd.concat(datamining_reg, axis = 1, join = 'outer')
            
        result = pd.merge(datamining_reg.reset_index(), competitors.reset_index(), how='left')
#         print('comp', result.head())
        result.to_sql('competitors', con = engine, index = False, if_exists = 'append')
        print('Succesfully uploaded competitors from {} to sql server'.format(regName))
        


       

In [ ]:
## TABLE 4 wind
# ,[regatta]
# ,[race]
# ,[timestamp_ms]
# ,[lat_deg]
# ,[lon_deg]
# ,[WindSpd_kts]
# ,[WindDir_deg]
# ,[dampened_WindSpd_kts]
# ,[dampened_WindDir_deg]


for serv in eventSelection:
    for indx, regatta in serv['regattas'].iterrows():
        regName = regatta.regatta
        dir_races =  parentdir + '/' + outdirbase + webAddressToFilename(serv['name']) + \
                    webAddressToFilename(regattaLoc(regName)) + '/races/'
        for race in os.listdir(dir_races):
            filepath = dir_races + race + "/wind--fromtime=2012-01-01T10__12__03Z&totime=2019-12-31T10__12__03Z.json"
            if filepath.endswith(".json"):
                with open(filepath) as json_file:
#                     print('file extracted: ', filepath)
                    data = json.load(json_file)
                dic_data = {'regatta': regName, 'race': filenameToWebAddress(race), 'data': data}
                df = cd.transformJsonToPD(dic_data, 'wind')#.set_index(['regatta', 'comp_name']) 
                df['timepoint-ms'] = df['timepoint-ms'].apply(int)
#                 print('df: ',df.head(), df.dtypes)
                try:
                    df.to_sql('wind', con = engine, index = False, if_exists = 'append')
                    print('Succesfully uploaded wind for {} race {}'.format(regName, race))
                except IntegrityError as e:
                    print('WARNING: Failed to upload to sql: '+ str(e))
               
            
            
#Tokyo 2019 - Laser Radial//races/R2 (Laser Radial)/wind--fromtime=2012-01-01T10__12__03Z&totime=2019-12-31T10__12__03Z.json                
# WARNING: Failed to upload to sql: (pyodbc.IntegrityError) ('23000', "[23000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Cannot insert duplicate key row in object 'dbo.wind' with unique index 'idx'. The duplicate key value is (Tokyo 2019 - Laser Radial, R2 (Laser Radial), 35.2943, 139.498, 1566027434659)
#file extracted:  X:\TU_Delft\2_Master\3_Stage\2_Code\SAPDataAnalysis//data/raw/tokyo2019/regattas/Tokyo 2019 - RS__X Men//races/R3 (RS__X Men)/wind--fromtime=2012-01-01T10__12__03Z&totime=2019-12-31T10__12__03Z.json
# WARNING: Failed to upload to sql: (pyodbc.IntegrityError) ('23000', "[23000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Cannot insert duplicate key row in object 'dbo.wind' with unique index 'idx'. The duplicate key value is (Tokyo 2019 - RS:X Men, R3 (RS:X Men), 35.2954, 139.495, 1566025905989)

In [ ]:
## TABLE 5 legs
# [regatta]
# ,[race]
# ,[leg_nr]
# ,[fromWaypointId]
# ,[toWaypointId]
# ,[to]
# ,[from]
# ,[upOrDownwindLeg]
# ,[leg_nr_from_finish]
# ,[distance]


for serv in eventSelection:
    for indx, regatta in serv['regattas'].iterrows():
        regName = regatta.regatta
        dir_races =  parentdir + '/' + outdirbase + webAddressToFilename(serv['name']) + \
                        webAddressToFilename(regattaLoc(regName)) + '/races/'
        for race in os.listdir(dir_races):
            filepath = dir_races + race + "/competitors/legs.json"
            if filepath.endswith(".json"):
                with open(filepath) as json_file:
#                     print('file extracted: ', filepath)
                    data = json.load(json_file)
                dic_data = {'regatta': regName, 'race': filenameToWebAddress(race), 'data': data}
                df = cd.transformJsonToPD(dic_data, 'legs') 
#                 print('df: ',df.head())

                try:
                    df.to_sql('legs', con = engine, index = False, if_exists = 'append')
                    print('Succesfully uploaded wind for regatta: \t{}, race: \t{}'.format(regName, race))
                except IntegrityError as e:
                    print('WARNING: Failed to upload to sql: '+ str(e))
               
  


In [ ]:
### TABLE 6
## This code create all entries in race_comp. We do this by checking that the nr_competitors in 
## the races table is still NULL because we did not update that yet.
## We will later add rank, based on comp_leg
cursor.execute("""
INSERT INTO race_comp(regatta, race, comp_id)
SELECT races.regatta, races.race, competitors.comp_id FROM races, competitors
WHERE races.regatta = competitors.regatta
 and races.nr_competitors IS NULL
""")


In [ ]:
## TABLE 7 comp_leg
# [regatta]
# ,[race]
# ,[leg_nr]
# ,[comp_id]
# ,[competitor_distanceTraveled-m]
# ,[competitor_averageSOG-kts]
# ,[competitor_tacks]
# ,[competitor_jibes]
# ,[competitor_penaltyCircles]
# ,[competitor_rank]
# ,[competitor_gapToLeader-s]
# ,[competitor_gapToLeader-m]
# ,[competitor_started]
# ,[competitor_finished]




for serv in eventSelection:
    
    for indx, regatta in serv['regattas'].iterrows():
        regName = regatta.regatta
        dir_races =  parentdir + '/' + outdirbase + webAddressToFilename(serv['name']) + \
                    webAddressToFilename(regattaLoc(regName)) + '/races/'
        for race in os.listdir(dir_races):
            filepath = dir_races + race + "/competitors/legs.json"
            if filepath.endswith(".json"):
                with open(filepath) as json_file:
#                     print('file extracted: ', filepath)
                    data = json.load(json_file)
                dic_data = {'regatta': regName, 'race': filenameToWebAddress(race), 'data': data}
                df = cd.transformJsonToPD(dic_data, 'comp_leg') 
#                 print('df: ',df.head())



                try:
                    df.to_sql('comp_leg', con = engine, index = False, if_exists = 'append')
                    print('Succesfully uploaded wind for regatta: \t{}, race: \t{}'.format(regName, race))
                except IntegrityError as e:
                    print('Could not add races of {} {} to the database because of IntegrityError: {}'.format(regName,
                                                                                                              race,
                                                                                                              e))


In [ ]:
### Markpassings, to update legnr_columnn in positions

cursor.execute("""
DROP TABLE temp_markpassings
CREATE TABLE temp_markpassings
(regatta varchar(50) NOT NULL,
race varchar(50) NOT NULL,
comp_id binary(16) NOT NULL,
leg_nr int NOT NULL,
begin_leg_ms bigint NULL,
end_leg_ms bigint NULL

)

CREATE UNIQUE INDEX idx ON temp_markpassings (regatta, race, leg_nr, comp_id);
CREATE CLUSTERED INDEX idx_2 ON temp_markpassings (regatta, race, comp_id);

""")

cursor.execute("""
ALTER TABLE temp_markpassings
ADD CONSTRAINT fk_temp_markpassings_comp_leg FOREIGN KEY (regatta, race, leg_nr, comp_id)
      REFERENCES comp_leg (regatta, race, leg_nr, comp_id)
      ON DELETE CASCADE
      ON UPDATE CASCADE
""")

In [13]:
#### TEMPORARY TABLE MARKPASSINGS
for serv in eventSelection:
    for indx, regatta in serv['regattas'].iterrows():
        regName = regatta.regatta
        dir_races =  parentdir + '/' + outdirbase + webAddressToFilename(serv['name']) +\
                    webAddressToFilename(regattaLoc(regName)) + '/races/'
        for race in os.listdir(dir_races):
            filepath = dir_races + race + "/markpassings.json"
            with open(filepath) as json_file:
#                 print('file extracted: ', filepath)
                data = json.load(json_file)
            dic_data = {'regatta': regName, 'race': filenameToWebAddress(race), 'data': data}
            df = cd.transformJsonToPD(dic_data, 'markpassings') 
            if not df.empty:
#                 print(df.leg_nr.unique())
                try:                
                    df.to_sql('temp_markpassings', con = engine, index = False, if_exists = 'append')

                    print('Succesfully added markpassings of \t{} \t{} \tto the database'.format(regName, race))
                except IntegrityError as e:
                    print('Could not add markpassings of \t{} \t{} \tto the database because of IntegrityError:'.format(regName,race))
            else:
                print('Warning: markpassings for \t{} \t{} \t was empty'.format(regName, race))



Succesfully added markpassings of 	Tokyo 2019 - 49er 	R1 (49er) 	to the database
Succesfully added markpassings of 	Tokyo 2019 - 49er 	R10 (49er) 	to the database
Succesfully added markpassings of 	Tokyo 2019 - 49er 	R11 (49er) 	to the database
Succesfully added markpassings of 	Tokyo 2019 - 49er 	R12 (49er) 	to the database
Succesfully added markpassings of 	Tokyo 2019 - 49er 	R13 Medal (49er) 	to the database
Succesfully added markpassings of 	Tokyo 2019 - 49er 	R2 (49er) 	to the database
Succesfully added markpassings of 	Tokyo 2019 - 49er 	R3 (49er) 	to the database
Succesfully added markpassings of 	Tokyo 2019 - 49er 	R4 (49er) 	to the database
Succesfully added markpassings of 	Tokyo 2019 - 49er 	R5 (49er) 	to the database
Succesfully added markpassings of 	Tokyo 2019 - 49er 	R6 (49er) 	to the database
Succesfully added markpassings of 	Tokyo 2019 - 49er 	R7 (49er) 	to the database
Succesfully added markpassings of 	Tokyo 2019 - 49er 	R8 (49er) 	to the database
Succesfully added 

Succesfully added markpassings of 	Tokyo 2019 - Finn 	R2 (Finn) 	to the database
Succesfully added markpassings of 	Tokyo 2019 - Finn 	R3 (Finn) 	to the database
Succesfully added markpassings of 	Tokyo 2019 - Finn 	R4 (Finn) 	to the database
Succesfully added markpassings of 	Tokyo 2019 - Finn 	R5 (Finn) 	to the database
Succesfully added markpassings of 	Tokyo 2019 - Finn 	R6 (Finn) 	to the database
Succesfully added markpassings of 	Tokyo 2019 - Finn 	R7 (Finn) 	to the database
Succesfully added markpassings of 	Tokyo 2019 - Finn 	R8 (Finn) 	to the database
Succesfully added markpassings of 	Tokyo 2019 - Finn 	R9 (Finn) 	to the database
Succesfully added markpassings of 	Tokyo 2019 - Nacra 17 	R1 (Nacra 17) 	to the database
Succesfully added markpassings of 	Tokyo 2019 - Nacra 17 	R10 (Nacra 17) 	to the database
Succesfully added markpassings of 	Tokyo 2019 - Nacra 17 	R11 (Nacra 17) 	to the database
Succesfully added markpassings of 	Tokyo 2019 - Nacra 17 	R12 (Nacra 17) 	to the da

In [14]:
## TABLE 8 positions
for serv in eventSelection:
    for indx, regatta in serv['regattas'].iterrows():
        regName = regatta.regatta
        dir_races =  parentdir + '/' + outdirbase + webAddressToFilename(serv['name']) + \
                    webAddressToFilename(regattaLoc(regName)) + '/races/'
        for race in os.listdir(dir_races):
            
            print('Get first pos', end = '\r')
            sql = """ SELECT TOP (1) * FROM positions
                    WHERE regatta = '{}'
                    and race = '{}'
                  """.format(regName, race)
            pos = pd.read_sql(sql, con = engine) ## already a positions for this race ?   
            if not pos.empty:
                print('Already positions found in db for \t{} \t{}'.format(regName, race))
                continue

            print('get markpassings', end="\r")
            sql = """ SELECT * FROM temp_markpassings
                    WHERE regatta = '{}'
                    and race = '{}'
                    """.format(regName, race)
            mps = pd.read_sql(sql, con = engine) ## markpassings for this race
            if mps.empty:
                print('No markpassints where found for\t{} \t{}'.format(regName, race))
                continue

            filepath = dir_races + race + "/competitors/positions.json"
            with open(filepath) as json_file:
        #                     print('file extracted: ', filepath)
                data = json.load(json_file)
            dic_data = {'regatta': regName, 'race': filenameToWebAddress(race), 'data': data}

            print('transform to pd', end="\r")
            df = cd.transformJsonToPD(dic_data, 'competitor_positions') 

            print('determine leg numbers', end="\r")
            for idx, comp_leg in mps.iterrows():
                df.loc[(df.comp_id == comp_leg.comp_id) & (df['timepoint_ms'] > comp_leg.begin_leg_ms)
                       & (df['timepoint_ms'] < comp_leg.end_leg_ms),'leg_nr'] = comp_leg.leg_nr
            df = df.dropna(subset=['leg_nr'])


            try:
                print('Uploading positions to db                        ', end="\r")
                df.to_sql('positions', con = engine, index = False, if_exists = 'append', chunksize = 10000)
                print('Added positions of \t{} \t{} \t succesfully to the database'.format(regName, race))
            except IntegrityError as e:
                print('Could not add positions of \t{} \t{} \tto the database because of IntegrityError.'.format(regName,race))


Added positions of 	Tokyo 2019 - 49er 	R1 (49er) 	 succesfully to the database
Added positions of 	Tokyo 2019 - 49er 	R10 (49er) 	 succesfully to the database
Added positions of 	Tokyo 2019 - 49er 	R11 (49er) 	 succesfully to the database
Added positions of 	Tokyo 2019 - 49er 	R12 (49er) 	 succesfully to the database
Added positions of 	Tokyo 2019 - 49er 	R13 Medal (49er) 	 succesfully to the database
Added positions of 	Tokyo 2019 - 49er 	R2 (49er) 	 succesfully to the database
Added positions of 	Tokyo 2019 - 49er 	R3 (49er) 	 succesfully to the database
Added positions of 	Tokyo 2019 - 49er 	R4 (49er) 	 succesfully to the database
Added positions of 	Tokyo 2019 - 49er 	R5 (49er) 	 succesfully to the database
Added positions of 	Tokyo 2019 - 49er 	R6 (49er) 	 succesfully to the database
Added positions of 	Tokyo 2019 - 49er 	R7 (49er) 	 succesfully to the database
Added positions of 	Tokyo 2019 - 49er 	R8 (49er) 	 succesfully to the database
Added positions of 	Tokyo 2019 - 49er 	R9 

Added positions of 	Tokyo 2019 - Finn 	R9 (Finn) 	 succesfully to the database
Added positions of 	Tokyo 2019 - Nacra 17 	R1 (Nacra 17) 	 succesfully to the database
Added positions of 	Tokyo 2019 - Nacra 17 	R10 (Nacra 17) 	 succesfully to the database
Added positions of 	Tokyo 2019 - Nacra 17 	R11 (Nacra 17) 	 succesfully to the database
Added positions of 	Tokyo 2019 - Nacra 17 	R12 (Nacra 17) 	 succesfully to the database
Added positions of 	Tokyo 2019 - Nacra 17 	R13 Medal (Nacra 17) 	 succesfully to the database
Added positions of 	Tokyo 2019 - Nacra 17 	R2 (Nacra 17) 	 succesfully to the database
Added positions of 	Tokyo 2019 - Nacra 17 	R3 (Nacra 17) 	 succesfully to the database
Added positions of 	Tokyo 2019 - Nacra 17 	R4 (Nacra 17) 	 succesfully to the database
Added positions of 	Tokyo 2019 - Nacra 17 	R5 (Nacra 17) 	 succesfully to the database
Added positions of 	Tokyo 2019 - Nacra 17 	R6 (Nacra 17) 	 succesfully to the database
Added positions of 	Tokyo 2019 - Nacra 17

### Extra functions in case something went wrong for a few entries

In [14]:
### Extra functions to manually add tracks of seperate competitors
## in the previous cell tracks have been added per race, but for some reason it went wrong in some cases

def uploadPositions(regName, race, comp_id):
    dir_races =  parentdir + '/' + outdirbase + webAddressToFilename(serv['name']) + \
                    webAddressToFilename(regattaLoc(regName)) + '/races/'
    filepath = dir_races + webAddressToFilename(race) + "/competitors/positions.json"
    sql = """ SELECT * FROM temp_markpassings
            WHERE regatta = '{}'
            and race = '{}'
            and comp_id = {}
            """.format(regName, race, comp_id)
    mps = pd.read_sql(sql, con = engine) ## markpassings for this race
    
    if mps.empty:
        print('No markpassings found, try to download them again')
    
    with open(filepath) as json_file:
#                     print('file extracted: ', filepath)
        data = json.load(json_file)
    dic_data = {'regatta': regName, 'race': filenameToWebAddress(race), 'data': data}

    print('transform to pd', end="\r")
    df = cd.transformJsonToPD(dic_data, 'competitor_positions') 

    print('determine leg numbers', end="\r")
    for idx, comp_leg in mps.iterrows():
        df.loc[(df.comp_id == comp_leg.comp_id) & (df['timepoint_ms'] > comp_leg.begin_leg_ms)
               & (df['timepoint_ms'] < comp_leg.end_leg_ms),'leg_nr'] = comp_leg.leg_nr
    df = df.dropna(subset=['leg_nr'])
#                 print(mps.head())
#                 print(df.head())

    try:
        print('Uploading positions to db', end="\r")
        df.to_sql('positions', con = engine, index = False, if_exists = 'append', chunksize = 10000)
        print('Added positions of \t{} \t{} \t succesfully to the database'.format(regName, race))
    except IntegrityError as e:
        print('Could not add positions of \t{} \t{} \tto the database because of IntegrityError.'.format(regName,race))
    return df


In [30]:
# quick import when a positions file was truncated 

import urllib, json, requests

servName = 'hwcs2020-round1'
regName = 'HWCS 2020 Round 1 - Finn'
raceName = 'R6 (FINN)'

url = 'https://www.sapsailing.com/sailingserver/api/v1/regattas/{}/races/{}/competitors/positions'.format(regName,
                                                                                                          raceName)
filename = '{}/data/raw/{}/regattas/{}/races/{}/competitors/positions.json'.format(parentdir, servName, regName, raceName)
try:
    r = requests.get(url).json()
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    print(filename)
except ValueError:
    print("Oops! ", loc , " did not contain the right data for the races")

with open(filename, 'w+') as outfile:
    print('Created new file:', filename)
    json.dump(r, outfile)

X:\TU_Delft\2_Master\3_Stage\2_Code\SAPDataAnalysis/data/raw/hwcs2020-round1/regattas/HWCS 2020 Round 1 - Finn/races/R6 (FINN)/competitors/positions.json
Created new file: X:\TU_Delft\2_Master\3_Stage\2_Code\SAPDataAnalysis/data/raw/hwcs2020-round1/regattas/HWCS 2020 Round 1 - Finn/races/R6 (FINN)/competitors/positions.json


# FILL COLUMNS TO COMPLETE DATABASE


## Update final rank in race_comp


In [ ]:
cursor.execute("""
UPDATE
    race_comp
SET
    race_comp.rank = (
            SELECT comp_leg.competitor_rank from comp_leg 
                    INNER JOIN legs ON comp_leg.regatta = legs.regatta
                                AND comp_leg.race = legs.race
                                AND comp_leg.leg_nr = legs.leg_nr
            WHERE legs.leg_nr_from_finish = 1
                AND race_comp.regatta = comp_leg.regatta 
                AND race_comp.race = comp_leg.race
                AND race_comp.comp_id = comp_leg.comp_id
)
""")


In [ ]:
cnxn.commit()

## Get chosen side of competitors for leg 1

In [14]:
def trackStatistics(track_df):
    #transform to equidistant field
    phi_0 = track_df['lat-deg'][0] ## latitude
    l_0 = track_df['lng-deg'][0] ## longitude
    x = np.array(cos(phi_0 * 2 * pi/360) * (track_df['lng-deg']- l_0))
    y = np.array( track_df['lat-deg']- phi_0)
    
    #rotate and scale to 0-1
    theta = -atan2(y[-1:],x[-1:])
    T = np.matrix([[cos(theta), -sin(theta)],[sin(theta), cos(theta)]])
    X = T*np.matrix([x,y])
    X = X/(X[0,-1:])
    
    #statistics
    avg_side = np.mean(X[1])/.25 * 100
    most_left = np.max(X[1]) /.5 * 100
    most_right = -np.min(X[1]) /.5 *100
    
    return avg_side, most_left, most_right


def coordinatesToEquiv(track_df, centre = True):
    if centre:
        phi_0 = track_df['lat-deg'][0] ## latitude
        l_0 = track_df['lng-deg'][0] ## longitude
    else:
        phi_0 = 35.160967
        l_0 = 139.309617
    x = np.array(cos(phi_0 * 2 * pi/360) * (track_df['lng-deg']- l_0))
    y = np.array( track_df['lat-deg']- phi_0)
    return np.array([x, y])



In [15]:
for indx, regatta in serv['regattas'].iterrows():
    regName = regatta.regatta

    sql = """SELECT  comp_leg.regatta, comp_leg.race, comp_leg.comp_id
                    ,comp_leg.competitor_started
                    ,comp_leg.competitor_finished
                    ,comp_leg.avg_side

            from comp_leg INNER JOIN regattas
            on regattas.regatta = comp_leg.regatta
            where regattas.regatta = '{}'
                  and leg_nr = 1""".format(regName)
    df = pd.read_sql_query(sql, con = engine)
    print("{}\n tracks which occured faults:".format(regName))       
    for index, row in df.iterrows():
        comp_id = "0x{}".format(row['comp_id'].hex().upper())

        sql = """SELECT  TOP (1) *
          FROM comp_leg
          WHERE regatta = '{}'
          and race = '{}'
          and comp_id = {}
          and leg_nr = 1""".format(row['regatta'] , row['race'] , comp_id)
        comp_leg = pd.read_sql_query(sql, con = engine, chunksize=None)

        if not (comp_leg.avg_side.values[0] is None):
#             print(row['regatta'], row['race'] , comp_id , end="\t")
#             print('Side already determined')
            continue

        if not row['competitor_started'] or not row['competitor_finished']:
#             print(row['regatta'], row['race'] , comp_id , end="\t")
#             print('comp did not start or finish leg')
            continue


        sql = """SELECT  *
          FROM positions
          WHERE regatta = '{}'
          and race = '{}'
          and comp_id = {}
          and leg_nr = 1""".format(row['regatta'] , row['race'] , comp_id)
        track = pd.read_sql_query(sql, con = engine, chunksize=None)

        if track.empty:
            print(row['regatta'], row['race'] , comp_id , end="\t")
            print('No track found, try to download it again')
            uploadPositions(row['regatta'], row['race'], comp_id)
            sql = """SELECT  *
              FROM positions
              WHERE regatta = '{}'
              and race = '{}'
              and comp_id = {}
              and leg_nr = 1""".format(row['regatta'] , row['race'] , comp_id)
            track = pd.read_sql_query(sql, con = engine, chunksize=None)
        if track.empty:
            print('No track found')
            break

        stats = [str(x) for x in trackStatistics(track)]
        cursor.execute("""
        UPDATE comp_leg
        SET    comp_leg.avg_side = {}
               ,comp_leg.most_left = {}
               ,comp_leg.most_right = {}
        WHERE
        regatta = '{}'
          and race = '{}'
          and comp_id = {}
          and leg_nr = 1
          """.format(stats[0], stats[1], stats[2], row['regatta'] , row['race'] , comp_id)
                      )
        cursor.commit()



WCS 2019 Enoshima - Laser
 tracks which occured faults:
WCS 2019 Enoshima - RS:X Men
 tracks which occured faults:
WCS 2019 Enoshima - 49er
 tracks which occured faults:
WCS 2019 Enoshima - Nacra 17
 tracks which occured faults:
WCS 2019 Enoshima - RS:X Women
 tracks which occured faults:
WCS 2019 Enoshima - 49erFX
 tracks which occured faults:
WCS 2019 Enoshima - Laser Radial
 tracks which occured faults:
WCS 2019 Enoshima - Finn
 tracks which occured faults:
WCS 2019 Enoshima - 470 Women
 tracks which occured faults:
WCS 2019 Enoshima - 470 Men
 tracks which occured faults:


In [ ]:
cnxn.commit()

## Determining Course Area

In [29]:
def getArea(race, areas):
    sql = """SELECT TOP (1) *
      FROM positions
      WHERE regatta = '""" + race['regatta'] + \
      """'and race = '"""+ race['race'] + \
      """'and leg_nr = 1"""
    track = pd.read_sql_query(sql, con = engine, index_col=None, coerce_float=True, params=None, parse_dates=None, chunksize=None)
    if track.empty:
        print('Track of race {} was empty'.format(race.race))
        return -1
    loc = coordinatesToEquiv(track, centre = False).reshape(1,2)
    areas['distance'] = areas.apply(lambda x: np.linalg.norm( x.equid -loc, ord = 1), axis =  1)
    area_id = areas.loc[areas[['distance']].idxmin(), 'id'].values[0]
    return area_id


In [28]:
sql = """SELECT  *
  FROM course_areas"""
areas = pd.read_sql_query(sql, con = engine)

areas['equid'] = areas.apply(lambda x:  coordinatesToEquiv(x, centre = False), axis =  1)


for serv in eventSelection:
    for indx, regatta in serv['regattas'].iterrows():
        regName = regatta.regatta
        sql = """SELECT  *
          FROM races
          WHERE races.regatta = '{}'""".format(regName)
        df = pd.read_sql_query(sql, con = engine, index_col=None, coerce_float=True, params=None, parse_dates=None, chunksize=None)

        for ind, race in df.iterrows():
            print('determining racearea of {} {}'.format(regName,race['race'])
                  , end = ' ')
            value = str(getArea(race, areas))
            if value == "-1":
                continue
            print(value)
            cursor.execute("""UPDATE races
                            SET races.course_area_id = {}
                            WHERE regatta = '{}'
                                AND race = '{}'
                            """.format(value, regName, race['race']))
            cursor.commit()
    



determining racearea of HWCS 2020 Round 1 - Laser M Medal (Laser Standard) Track of race regatta                               HWCS 2020 Round 1 - Laser
race                                   M Medal (Laser Standard)
raceShort                                                     M
raceId                 b'\xf2\xde\xb2@\xaep\x017$/\x06w?\x91rv'
maxWindSpd_kts                                              NaN
minWindSpd_kts                                              NaN
avgWindDir_deg                                              NaN
firstlegbearing_deg                                      201.88
course_area_id                                             None
nr_competitors                                             None
Name: 0, dtype: object was empty
determining racearea of HWCS 2020 Round 1 - Laser R08 (Laser Standard) 2
determining racearea of HWCS 2020 Round 1 - Laser R09 (Laser Standard) 2
determining racearea of HWCS 2020 Round 1 - Laser R1 (Laser Standard) 2
determining racearea

## Number of competitors per race and their relative rank

In [ ]:
cursor.execute("""
                UPDATE
                    races
                SET
                    races.nr_competitors = temp_table.nr_competitors
                FROM
                    (SELECT race_comp.regatta, race_comp.race, count(rank) as nr_competitors
                                              FROM race_comp 
                                              GROUP by regatta, race) AS temp_table
                    INNER JOIN races on temp_table.regatta = races.regatta
                            and temp_table.race = races.race
            """)
cursor.commit()

In [ ]:
cursor.execute("""
                UPDATE
                    comp_leg
                SET
                    comp_leg.rel_rank = temp_table.rel_rank
                FROM
                    (SELECT comp_leg.regatta, comp_leg.race, comp_leg.comp_id
                           ,comp_leg.competitor_rank
                           , comp_leg.competitor_rank/convert(decimal(4,2), races.nr_competitors) as rel_rank
                        FROM comp_leg INNER JOIN races
                            ON comp_leg.regatta = races.regatta 
                            and comp_leg.race = races.race
                        )
                        AS temp_table INNER JOIN comp_leg
                            ON comp_leg.regatta = temp_table.regatta 
                                 and comp_leg.race = temp_table.race
                                 and comp_leg.comp_id = temp_table.comp_id

                """)
cursor.commit()

## Calculate Correlations

In [39]:
## not yet right for other legs then 1, because avg side needs to be calculated first
for serv in eventSelection:
    for indx, regatta in serv['regattas'].iterrows():
        regName = regatta.regatta

        sql = """
        SELECT * from legs
        where leg_nr = 1
            and regatta = '{}'
            """.format(regName)
        
        legs = pd.read_sql(sql, engine)

        for index, row in legs.iterrows():
            print(row['regatta'], row['race'], row['leg_nr'])

            sql = """
            SELECT * from comp_leg
            WHERE regatta = '{}'
            and race = '{}'
            and leg_nr = {}
            """.format( row['regatta'],
                        row['race'],
                        row['leg_nr'])

            df = pd.read_sql(sql, engine)
            corrs = df.corr().rel_rank
            corrs = corrs.replace(np.nan, "NULL", regex=True)  ### replace NaN's by NULL so they can be updated in the sql df
            try:
                cursor.execute("""
                UPDATE legs
                SET    legs.correlation_tacks = {}
                      ,legs.correlation_side = {}
                      ,legs.correlation_avgSOG = {}
                      ,legs.correlation_traveledDistance = {}
                      ,legs.correlation_jibes = {}
                WHERE
                regatta = '{}'
                and race = '{}'
                and leg_nr = '{}'
                  """.format(corrs['competitor_tacks'],
                             corrs['avg_side'],
                             corrs['competitor_averageSOG-kts'],
                             corrs['competitor_distanceTraveled-m'],
                             corrs['competitor_jibes'],                 
                             row['regatta'],
                             row['race'],
                             row['leg_nr']))
            except KeyError:
                print('Key error for correlations')
            cursor.commit()


HWCS 2020 Round 1 - Laser M Medal (Laser Standard) 1
Key error for correlations
HWCS 2020 Round 1 - Laser R08 (Laser Standard) 1
HWCS 2020 Round 1 - Laser R09 (Laser Standard) 1
HWCS 2020 Round 1 - Laser R1 (Laser Standard) 1
HWCS 2020 Round 1 - Laser R10 (Laser Standard) 1
HWCS 2020 Round 1 - Laser R2 (Laser Standard) 1
HWCS 2020 Round 1 - Laser R3 (Laser Standard) 1
HWCS 2020 Round 1 - Laser R4 (Laser Standard) 1
HWCS 2020 Round 1 - Laser R5 (Laser Standard) 1
HWCS 2020 Round 1 - Laser R6 (Laser Standard) 1
HWCS 2020 Round 1 - Laser R7 (Laser Standard) 1
HWCS 2020 Round 1 - Laser Radial M Medal (Laser Radial) 1
Key error for correlations
HWCS 2020 Round 1 - Laser Radial R08 (Laser Radial) 1
HWCS 2020 Round 1 - Laser Radial R09 (Laser Radial) 1
HWCS 2020 Round 1 - Laser Radial R1 (Laser Radial) 1
HWCS 2020 Round 1 - Laser Radial R10 (Laser Radial) 1
HWCS 2020 Round 1 - Laser Radial R2 (Laser Radial) 1
HWCS 2020 Round 1 - Laser Radial R3 (Laser Radial) 1
HWCS 2020 Round 1 - Laser Radia

In [5]:
## Plot table positions
alreadydone = ['HWCS 2020 Round 1 - Laser',
               'HWCS 2020 Round 1 - Laser Radial',
               'HWCS 2020 Round 1 - 49erFX',
               'HWCS 2020 Round 1 - Nacra 17',
               'HWCS 2020 Round 1 - 470 Men',
               'HWCS 2020 Round 1 - RS:X Men',
               'HWCS 2020 Round 1 - Finn',
               'HWCS 2020 Round 1 - RS:X Women',
               'HWCS 2020 Round 1 - 49er',
               'HWCS 2020 Round 1 - 470 Women',
               'WCS 2019 Enoshima - Laser',
               'WCS 2019 Enoshima - RS:X Men']

for serv in eventSelection:
    for indx, regatta in serv['regattas'].iterrows():
        regName = regatta.regatta
        if regName in alreadydone:
            print('{} already done'.format(regName))
            continue
        sql = """
                INSERT INTO positions_plot
                SELECT * FROM positions
                WHERE positions.regatta = '{}'
                AND positions.timepoint_ms % 20000 = 0
                """.format(regName)
        cursor.execute(sql)
        cursor.commit()
        print(regName, ' done ')

HWCS 2020 Round 1 - Laser already done
HWCS 2020 Round 1 - Laser Radial already done
HWCS 2020 Round 1 - 49erFX already done
HWCS 2020 Round 1 - Nacra 17 already done
HWCS 2020 Round 1 - 470 Men already done
HWCS 2020 Round 1 - RS:X Men already done
HWCS 2020 Round 1 - Finn already done
HWCS 2020 Round 1 - RS:X Women already done
HWCS 2020 Round 1 - 49er already done
HWCS 2020 Round 1 - 470 Women already done
WCS 2019 Enoshima - Laser already done
WCS 2019 Enoshima - RS:X Men already done
WCS 2019 Enoshima - 49er  done 
WCS 2019 Enoshima - Nacra 17  done 
WCS 2019 Enoshima - RS:X Women  done 
WCS 2019 Enoshima - 49erFX  done 
WCS 2019 Enoshima - Laser Radial  done 
WCS 2019 Enoshima - Finn  done 
WCS 2019 Enoshima - 470 Women  done 
WCS 2019 Enoshima - 470 Men  done 


In [ ]:
CREATE VIEW positions_view
WITH SCHEMABINDING
AS 
SELECT
    r.regatta,
    rc.race,
    l.leg_nr,
    c.comp_id,
    timepoint_ms,
    [lat-deg],
	[lng-deg]
	,[speed-kts]
	,[truebearing-deg]
FROM
    dbo.positions p
INNER JOIN dbo.competitors c
    ON c.comp_id = p.comp_id
	 and c.regatta = p.regatta
INNER JOIN dbo.legs l
	ON l.regatta = p.regatta
	 and l.race = p.race
	 and l.leg_nr = p.leg_nr
INNER JOIN dbo.races rc 
    ON rc.regatta = p.regatta
	 and rc.race = p.race
INNER JOIN dbo.regattas r
	ON r.regatta = p.regatta
WHERE l.leg_nr = 1
 and p.timepoint_ms % 20000 = 0
	 ;

### Usefull other queries

In [ ]:
cursor.execute("""

USE nerinedatabase;
EXEC sp_spaceused;
""")


In [ ]:
cursor.execute("""
DROP INDEX idx ON positions;
CREATE UNIQUE CLUSTERED INDEX idx ON positions (regatta, race, comp_id, timepoint_ms ASC);

""")


In [ ]:
# info over indeces
EXEC sp_helpindex '[[[SCHEMA-NAME.TABLE-NAME]]]'
GO

## snelle manier om aantal rijen in tabel te krijgen
DECLARE @TableName sysname
SET @TableName = 'positions'
 
SELECT OBJECT_NAME(object_id), SUM(row_count) AS rows
FROM sys.dm_db_partition_stats
WHERE object_id = OBJECT_ID(@TableName)
AND index_id < 2
GROUP BY OBJECT_NAME(object_id);